# Pick and Place 
## Mécanique

Pour un moteur avec un couple donné 𝑇, le rayon effectif 𝑟 et le rendement 𝜂,la force maximale 𝐹, est donnée par :

$F=T⋅η/r$

Ensuite, en connaissant la force 𝐹 nous pouvons calculer la masse totale $m_{total}$ déplacée en fonction des accélérations et des résistances. 

$$m_{total}=m_{c}+m_{a}$$
Horizontal :

$$F=m_{total}⋅a+μ⋅m_{total}⋅g$$

Résolvons pour $m_{total}$ :

$$m_{total}= \frac{F}{a+μ⋅g}$$

Vertical : 

$$F=m_{total}⋅(a+g)+μ⋅m_{total}⋅g$$

Résolvons pour $m_{total}$ :

$$m_{total}= \frac{F}{a+g+μ⋅g}$$



---

### Calcul du couple nécessaire

In [6]:
def calcul_force(m_c, m_a, a, v, mu, g=9.81, r=0.01, eta=0.9, ratio=1.0, axe='horizontal'):
    """
    -param m_c: Masse du composant manipulé (kg)
    -param m_a: Masse des parties mobiles de l'axe (kg)
    -param a: Accélération maximale (m/s^2)
    -param v: Vitesse cible (m/s)
    -param mu: Coefficient de friction (sans unité)
    -param g: Accélération gravitationnelle (m/s^2)
    -param r: Rayon effectif du moteur ou poulie (m)
    -param eta: Rendement mécanique (0 à 1)
    -param ratio: Ratio du jeu d'engrenage (sans unité, >1 pour démultiplication)
    -param axe: 'horizontal' ou 'vertical'
    -return: Force linéaire (N), Couple (Nm), Vitesse ajustée (m/s)
    """
    m_total = m_c + m_a

    # Calcul de la force selon l'axe
    if axe == 'horizontal':
        force = m_total * a + mu * m_total * g
    elif axe == 'vertical':
        force = m_total * (a + g) + mu * m_total * g
    else:
        raise ValueError("L'axe doit être 'horizontal' ou 'vertical'")

    # Prise en compte du ratio pour le couple
    couple = (force * r / eta) / ratio

    # Ajustement de la vitesse de sortie en fonction du ratio
    vitesse_sortie = v / ratio

    return force, couple, vitesse_sortie

# Exemple d'utilisation
m_c = 0.1  # Masse du composant (kg)
m_a = 4.9  # Masse des parties mobiles (kg)
a = 50.0     # Accélération (m/s^2)
v = 2        # Vitesse (m/s)
mu = 0.1     # Coefficient de friction
r = 0.01146  # Rayon de poulie (m)
eta = 1      # Rendement (0,9 = 90%)
ratio = 2    # Ratio du jeu d'engrenage
coef_secu = 1.2  # Coefficient de sécurité marge

force, couple, vitesse_sortie = calcul_force(m_c, m_a, a, v, mu, r=r, eta=eta, ratio=ratio, axe='horizontal')
print(f"Force: {force:.2f} N, Couple: {couple:.2f} Nm, Vitesse ajustée: {vitesse_sortie:.2f} m/s")
print(f"Force marge: {force*coef_secu:.2f} N, Couple marge: {couple*coef_secu:.2f} Nm")


Force: 254.91 N, Couple: 1.46 Nm, Vitesse ajustée: 1.00 m/s
Force marge: 305.89 N, Couple marge: 1.75 Nm


---

### Calcul de la charge

In [10]:
def calcul_masse_maximale(T, r, eta, a,ratio, mu, axe='horizontal', g=9.81):
    """    
    :param T: Couple du moteur (Nm)
    :param r: Rayon effectif de la poulie ou de l'axe (m)
    :param eta: Rendement mécanique (0 à 1)
    :param a: Accélération maximale (m/s^2)
    :param mu: Coefficient de friction (sans unité)
    :param axe: 'horizontal' ou 'vertical'
    :param g: Accélération gravitationnelle (m/s^2)
    :return: Masse maximale totale (kg)
    """
    # Calcul de la force maximale disponible
    F = (T * eta / r) * ratio  # Force en newtons (N)

    # Calcul de la masse maximale selon l'axe
    if axe == 'horizontal':
        m_total = F / (a + mu * g)
    elif axe == 'vertical':
        m_total = F / (a + g + mu * g)
    else:
        raise ValueError("L'axe doit être 'horizontal' ou 'vertical'")
    
    return m_total

# Exemple d'utilisation
T = 3.0      # Couple du moteur en Nm
r = 0.01146     # Rayon de la poulie (1 cm)
eta = 1    # Rendement mécanique (90%)
a = 50.0      # Accélération maximale (m/s^2)
mu = 0.1    # Coefficient de friction
ratio = 2    # Ratio du jeu d'engrenage

# Masse maximale pour un axe vertical
m_max_vertical = calcul_masse_maximale(T, r, eta, a, ratio, mu, axe='vertical')
print(f"Masse maximale (axe vertical) : {m_max_vertical:.2f} kg")
print(f"Masse maximale marge(axe vertical) : {m_max_vertical/coef_secu:.2f} kg")

# Masse maximale pour un axe horizontal
m_max_horizontal = calcul_masse_maximale(T, r, eta, a, ratio, mu, axe='horizontal')
print(f"Masse maximale (axe horizontal) : {m_max_horizontal:.2f} kg")
print(f"Masse maximale marge(axe horizontal) : {m_max_horizontal/coef_secu:.2f} kg")


Masse maximale (axe vertical) : 8.61 kg
Masse maximale marge(axe vertical) : 7.18 kg
Masse maximale (axe horizontal) : 10.27 kg
Masse maximale marge(axe horizontal) : 8.56 kg


---

In [17]:
def calcul_masse_maximale_vitesse(T, r, eta, a, v, ratio, mu_0, k=0, axe='horizontal', g=9.81):
    """
    Calcule la masse totale maximale (composant + parties mobiles) qu'un moteur peut déplacer,
    en tenant compte de la vitesse et du frottement dépendant de la vitesse.
    
    :param T: Couple du moteur (Nm)
    :param r: Rayon effectif de la poulie ou de l'axe (m)
    :param eta: Rendement mécanique (0 à 1)
    :param a: Accélération maximale (m/s^2)
    :param v: Vitesse cible (m/s)
    :param mu_0: Coefficient de friction statique
    :param k: Coefficient de friction dépendant de la vitesse (par défaut 0)
    :param axe: 'horizontal' ou 'vertical'
    :param g: Accélération gravitationnelle (m/s^2)
    :return: Masse maximale totale (kg), Puissance requise (W)
    """
    # Calcul de la force maximale disponible
    F = ( T * eta / r ) * ratio # Force en newtons (N)

    # Frottement dépendant de la vitesse
    mu_v = mu_0 + k * v

    # Calcul de la masse maximale selon l'axe
    if axe == 'horizontal':
        m_total = F / (a + mu_v * g)
    elif axe == 'vertical':
        m_total = F / (a + g + mu_v * g)
    else:
        raise ValueError("L'axe doit être 'horizontal' ou 'vertical'")

    # Calcul de la puissance nécessaire pour maintenir la vitesse
    P = F * v  # Puissance en watts (W)

    return m_total, P

# Exemple d'utilisation
T = 3.0      # Couple du moteur en Nm
r = 0.01146     # Rayon de la poulie (1 cm)
eta = 0.9    # Rendement mécanique (90%)
a = 30      # Accélération maximale (m/s^2)
v = 2      # Vitesse cible (m/s)
mu_0 = 0.1  # Coefficient de friction statique
k = 0     # Variation du frottement avec la vitesse
ratio = 2    # Ratio du jeu d'engrenage

# Masse maximale et puissance pour un axe vertical
m_max_vertical, P_vertical = calcul_masse_maximale_vitesse(T, r, eta, a, v, ratio, mu_0, k, axe='vertical')
print(f"Masse maximale (axe vertical) : {m_max_vertical:.2f} kg, Puissance : {P_vertical:.2f} W")

# Masse maximale et puissance pour un axe horizontal
m_max_horizontal, P_horizontal = calcul_masse_maximale_vitesse(T, r, eta, a, v, ratio, mu_0, k, axe='horizontal')
print(f"Masse maximale (axe horizontal) : {m_max_horizontal:.2f} kg, Puissance : {P_horizontal:.2f} W")


Masse maximale (axe vertical) : 11.55 kg, Puissance : 942.41 W
Masse maximale (axe horizontal) : 15.21 kg, Puissance : 942.41 W


---

## Calculs de vitesse maximale du moteur et distance parcourue par pas


1. Vitesse de rotation maximale ($𝜔_{max}$ en rad/s et RPM)

La vitesse de rotation maximale d'un moteur dépend de la fréquence d'entrée maximale du driver ($𝑓_{max}$) et du nombre de pas par révolution ($𝑁_{steps}$):

- En radians par seconde ($𝜔_{max}$):

$$𝜔_{max}= \frac{2π⋅𝑓_{max}}{𝑁_{steps}}$$

- En tours par minute (RPM):

$$RPM_{max}= \frac{𝑓_{max}⋅60}{𝑁_{steps}}$$

2. Vitesse linéaire et distance parcourue par pas (précision PnP)

- Vitesse linéaire ($v_{lin}$)

$$v_{lin}=ω⋅r$$

- Distance parcourue par pas ($d_{step}$)

$$d_{step}= \frac{π⋅D}{𝑁_{steps}}$$

Fréquence max des stepper moteur : https://www.omc-stepperonline.com/fr/support/quelle-est-la-vitesse-maximale-frequence-la-plus-elevee-du-moteur-pas-a-pas?srsltid=AfmBOoqeKuFIeAo0ru7t6FeIFOOswYjshOpxPyBiXgGPo6L8qANc8rjO

In [2]:
import math

# Fonction pour calculer la vitesse de rotation maximale
def vitesse_rotation_max(f_max):
    """
    Calcule la vitesse maximale de rotation du moteur.
    
    Paramètres :
    - f_max : Fréquence maximale du driver (en Hz)
    - N_steps : Nombre de pas par révolution (en steps/rev)
    
    Retourne :
    - vitesse_rad_s : Vitesse en rad/s
    - vitesse_rpm : Vitesse en tours par minute (RPM)
    """
    # Vitesse angulaire en rad/s
    vitesse_rad_s = (2 * math.pi * f_max)
    # Vitesse en tours par minute
    vitesse_rpm = (f_max * 60)
    return vitesse_rad_s, vitesse_rpm

# Fonction pour calculer la vitesse linéaire et la distance parcourue par step
def vitesse_lineaire_et_distance(omega, rayon_poulie, ratio, N_steps, diametre_poulie):
    """
    Calcule la vitesse linéaire et la distance parcourue par pas.
    
    Paramètres :
    - omega : Vitesse angulaire (en rad/s)
    - rayon_poulie : Rayon de la poulie (en mètres)
    - N_steps : Nombre de pas par révolution
    - diametre_poulie : Diamètre de la poulie (en mètres)
    
    Retourne :
    - vitesse_lin : Vitesse linéaire (en m/s)
    - distance_par_step : Distance parcourue par pas (en mètres)
    """
    # Vitesse linéaire
    vitesse_lin = ( omega * rayon_poulie ) / ratio
    # Distance parcourue par step
    distance_par_step = (math.pi * diametre_poulie) / N_steps
    distance_par_step /= ratio
    return vitesse_lin, distance_par_step

# Fonction pour calculer le nombre de step pour un tour en fonction du microstepping
def calculer_nombre_steps(N_steps_base, microstepping):
    """
    Calcule le nombre total de pas pour une révolution en fonction du microstepping.
    
    Paramètres :
    - N_steps_base : Nombre de pas par révolution sans microstepping (ex: 200 pas/rev)
    - microstepping : Facteur de microstepping (ex: 1, 2, 4, 8, 16, ...)
    
    Retourne :
    - N_steps : Nombre total de pas par révolution
    """
    return N_steps_base * microstepping


# Exemple d'utilisation
if __name__ == "__main__":

    # Paramètres
    f_max = 16.666  # Fréquence max du driver (en Hz) (25 Hz=1500RPM, 16.666Hz=1000RPM)
    N_steps_base = 200 # Nombre de pas par révolution sans microstepping (ex: 200 pas/rev) (ex = 1.8°/rev => 200 pas/rev)
    microstepping = 32 # Facteur de microstepping (ex: 1, 2, 4, 8, 16, 32 ...)
    N_steps = 0   # Nombre de pas par révolution (en microstepping 1/16)
    rayon_poulie = 0.01146 # Rayon de la poulie (en mètres)
    diametre_poulie = 2 * rayon_poulie  # Diamètre de la poulie (en mètres)
    ratio = 1    # Ratio du jeu d'engrenage 

    N_steps =calculer_nombre_steps(N_steps_base, microstepping)
    print(f"Nombre de step par révolution : {N_steps:.2f}")
    
    # Calculer la vitesse de rotation maximale
    vitesse_rad_s, vitesse_rpm = vitesse_rotation_max(f_max)
    print(f"Vitesse angulaire max : {vitesse_rad_s:.2f} rad/s")
    print(f"Vitesse max en RPM : {vitesse_rpm:.2f} tours/min")
    
    # Calculer la vitesse linéaire et la distance par pas
    vitesse_lin, distance_par_step = vitesse_lineaire_et_distance(vitesse_rad_s, rayon_poulie, ratio, N_steps, diametre_poulie)
    print(f"Vitesse linéaire max : {vitesse_lin*1000:.10f} mm/s")
    print(f"Distance parcourue par step : {distance_par_step*1000:.6f} mm -> {distance_par_step*1000000:.6f}  µm")


Nombre de step par révolution : 6400.00
Vitesse angulaire max : 104.72 rad/s
Vitesse max en RPM : 999.96 tours/min
Vitesse linéaire max : 1200.0403901356 mm/s
Distance parcourue par step : 0.011251 mm -> 11.250829  µm


---

### Bonus: calculs poulie et rapport d'engrenage

In [118]:
def calculer_engrenage(module=None, pas=None, nombre_dents=None, diametre=None):
    """
    Calcule les informations d'une poulie/engrenage (module, pas, nombre de dents, diamètre primitif).
    Il faut fournir au moins deux paramètres pour calculer le troisième.
    
    Paramètres :
    - module : Module de l'engrenage (en mm)
    - pas : Pas de l'engrenage/poulie (en mm, distance entre deux dents consécutives)
    - nombre_dents : Nombre de dents de l'engrenage/poulie
    - diametre : Diamètre primitif de l'engrenage/poulie (en mm)
    
    Retourne :
    - Un dictionnaire contenant les valeurs calculées
    """
    if module is None and pas is None:
        raise ValueError("Il faut fournir au moins un module ou un pas pour effectuer le calcul.")
    
    if module is None:
        # Si le module n'est pas fourni, le calculer à partir du pas
        module = pas
    
    elif pas is None:
        # Si le pas n'est pas fourni, le calculer à partir du module
        pas = module

    # Calculs basés sur les paramètres fournis
    if nombre_dents is None:
        # Si le nombre de dents n'est pas fourni, le calculer
        if diametre is not None:
            nombre_dents = int(diametre / module)
        else:
            raise ValueError("Il faut fournir le diamètre ou le nombre de dents pour effectuer le calcul.")
    elif diametre is None:
        # Si le diamètre n'est pas fourni, le calculer
        if nombre_dents is not None:
            diametre = round((module * nombre_dents)/math.pi,2)
        else:
            raise ValueError("Il faut fournir le diamètre ou le nombre de dents pour effectuer le calcul.")

    return {
        "module": module,
        "pas": pas,
        "nombre_dents": nombre_dents,
        "diametre": diametre
    }


def calculer_rapport_transmission(nombre_dents1=None, nombre_dents2=None, diametre1=None, diametre2=None, pas1=None, pas2=None, module1=None, module2=None):
    """
    Calcule le rapport de transmission entre deux poulies/engrenages avec possibilité de prendre en compte le module ou le pas.
    
    Paramètres :
    - nombre_dents1 : Nombre de dents du premier engrenage
    - nombre_dents2 : Nombre de dents du deuxième engrenage
    - diametre1 : Diamètre primitif du premier engrenage (en mm)
    - diametre2 : Diamètre primitif du deuxième engrenage (en mm)
    - pas1 : Pas de la première poulie (en mm)
    - pas2 : Pas de la deuxième poulie (en mm)
    - module1 : Module du premier engrenage (en mm)
    - module2 : Module du deuxième engrenage (en mm)
    
    Retourne :
    - Le rapport de transmission (R)
    """
    if nombre_dents1 is not None and nombre_dents2 is not None:
        # Calculer à partir des nombres de dents
        rapport = nombre_dents2 / nombre_dents1
    elif diametre1 is not None and diametre2 is not None:
        # Calculer à partir des diamètres primitifs
        rapport = diametre2 / diametre1
    elif pas1 is not None and pas2 is not None:
        # Calculer à partir des pas
        rapport = pas2 / pas1
    elif module1 is not None and module2 is not None:
        # Calculer à partir des modules
        rapport = module2 / module1
    else:
        raise ValueError("Au moins les nombres de dents, les diamètres, les pas ou les modules des deux engrenages sont nécessaires.")
    
    return rapport


# Exemple d'utilisation avec module ou pas
if __name__ == "__main__":
    # Exemple 1 : Calcul des caractéristiques d'un engrenage/poulie avec module ou pas
    engrenage1 = calculer_engrenage(pas=2, nombre_dents=16)
    print("Engrenage 1 (avec module) : ", engrenage1)
    
    engrenage2 = calculer_engrenage(pas=2, nombre_dents=36)
    print("Engrenage 2 (avec pas) : ", engrenage2)
    
    # Exemple 2 : Calcul du rapport de transmission avec module ou pas
    rapport1 = calculer_rapport_transmission(nombre_dents1=32, nombre_dents2=16)
    print(f"Rapport de transmission (Z2/Z1) : {rapport1:.2f}")
    
    # rapport2 = calculer_rapport_transmission(diametre1=40, diametre2=80)
    # print(f"Rapport de transmission (D2/D1) : {rapport2:.2f}")
    
    # rapport3 = calculer_rapport_transmission(pas1=2, pas2=4)
    # print(f"Rapport de transmission (Pas2/Pas1) : {rapport3:.2f}")
    
    # rapport4 = calculer_rapport_transmission(module1=2, module2=4)
    # print(f"Rapport de transmission (Module2/Module1) : {rapport4:.2f}")


Engrenage 1 (avec module) :  {'module': 2, 'pas': 2, 'nombre_dents': 16, 'diametre': 10.19}
Engrenage 2 (avec pas) :  {'module': 2, 'pas': 2, 'nombre_dents': 36, 'diametre': 22.92}
Rapport de transmission (Z2/Z1) : 0.50
